In [1]:
import json
from datetime import date, datetime, timedelta
import pandas as pd
import numpy as np    

## Some bot & human accouts from our advisor

In [121]:
bot_account=['edgarwinston','griffinhamza','jacksonjimmy','1ffyqhg4rmbk','powellernest','2rezoaf4bhly','nckj42dit5sb','scottphillip','oepa252sdx4p','myh2o4wayvxg']
human_account=['g44dinjygene','onebrother11','rvrkingfishr','iloveyoudapp','huiyong12345','pketothemoon','zhshj1212123','vipgamedice2','dldldldldldl','pkeniubixxxx']

# dicex

## transfer .json into .csv

In [2]:
dicex_dataset=pd.read_csv('dicex_data.csv',index_col=0)
del dicex_dataset['data']
dicex_dataset.dropna(axis=0,how='any',inplace=True)
dicex_dataset.head()
dicex_dataset.to_csv('dicex_data_new.csv')

C:\Users\sissy\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [122]:
# save labeled accounts again for future use
bot_account_new = list(np.unique(dicex_dataset[dicex_dataset.d_from.isin(bot_account)]['d_from']))
human_account_new = list(np.unique(dicex_dataset[dicex_dataset.d_from.isin(human_account)]['d_from']))
print(bot_account_new)
print(human_account_new)

['1ffyqhg4rmbk', 'edgarwinston', 'griffinhamza', 'nckj42dit5sb']
[]


In [32]:
#filter: find labeled accounts and put them into our training set
dicex_feature = dicex_dataset[dicex_dataset.d_from.isin(bot_account_new)| dicex_dataset.d_from.isin(human_account_new)].reset_index(drop=True)

In [33]:
#drop useless columns
dicex_dataset.drop(['name','trx_id','global_action_seq','trx_timestamp_unix','block_num','account_action_seq','d_memo'],axis=1,inplace=True)
#dicex_feature.head()

In [31]:
dicex = pd.DataFrame() # dicex is our final data set

# Add features

In [34]:
# add feature: num_account_type
df = dicex_dataset.groupby(['d_from','account']).count()['trx_timestamp'].reset_index().pivot('d_from','account','trx_timestamp').isna()
for type in np.unique(dicex_dataset.account):
    df[type] = df[type].apply(lambda x: 0 if x is True else 1)
df['num_account'] = df.apply(lambda x: x.sum(), axis=1)
dicex['num_account_type'] = list(df['num_account'])
dicex['user'] = df.index
dicex.index = df.index

In [25]:
# add feature: num_d_unit
df = dicex_dataset.groupby(['d_from','d_quantity_unit']).count()['trx_timestamp'].reset_index().pivot('d_from','d_quantity_unit','trx_timestamp').isna()
for type in np.unique(dicex_dataset.d_quantity_unit):
    df[type] = df[type].apply(lambda x: 0 if x is True else 1)
df['num_d_unit'] = df.apply(lambda x: x.sum(), axis=1)
dicex['num_d_unit'] = list(df['num_d_unit'])

In [26]:
# add feature: type_d_to
df = dicex_dataset.groupby(['d_from','d_to']).count()['trx_timestamp'].reset_index().pivot('d_from','d_to','trx_timestamp').isna()
for type in np.unique(dicex_dataset.d_to):
    df[type] = df[type].apply(lambda x: 0 if x is True else 1)
df['type_d_to'] = df.apply(lambda x: x.sum(), axis=1)
dicex['type_d_to'] = list(df['type_d_to'])

In [27]:
# add feature: Quantity_variance
df = dicex_dataset.groupby(['d_from','Day']).sum()['d_quantity'].reset_index().groupby('d_from').agg({'d_quantity':np.var})
dicex['quantity_variance'] = list(df['d_quantity'].apply(lambda x: 0 if np.isnan(x) else x))

In [28]:
# add feature: Frequency_variance
df = dicex_dataset.groupby(['d_from','Day']).count()['trx_timestamp'].reset_index().groupby('d_from').agg({'trx_timestamp':np.var})
dicex['frequency_variance'] = list(df['trx_timestamp'].apply(lambda x: 0 if np.isnan(x) else x))

In [51]:
# add feature: Time_interval_rate: 100 * reciprocal of min interval between two transaction
def find_min_interval(data):
    interval = []
    for i in range(len(data)-1):
        d = abs(data[i]-data[i+1])
        interval.append(d)
    return min(interval).total_seconds() if interval else float('inf')

interval = []
for x in np.unique(dicex_dataset.d_from):
    df = dicex_dataset[dicex_dataset.d_from==x].sort_values(by='trx_timestamp',ascending=False)['trx_timestamp'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f")).reset_index(drop=True)
    interval.append(100/(find_min_interval(df)+1)) # avoid time_interval=0
dicex['Time_interval_rate'] = interval

In [80]:
# add feature:   Activity_day(how many days each account is active for transactions in one month count/set/day)
# dicex_feature.groupby('d_from')['Day'].unique().reset_index() 
df = dicex_dataset.groupby('d_from')['Day'].nunique().reset_index() 
df.index = df['d_from']
dicex['activity_day'] = df['Day']

In [82]:
# add feature:  Active_hour(how many hours each account is active for transactions per day)
df = dicex_dataset.groupby(['d_from','Day'])['Hour'].nunique().reset_index() 
df1 = df.groupby("d_from")['Hour'].mean().reset_index() 
df1.index = df1['d_from']
dicex['activity_hour'] = df1['Hour']

In [84]:
# add feature: Max_Active_hour(how many hours each account is active for transactions in unit period)
df = dicex_dataset.groupby(['d_from','Day'])['Hour'].nunique().reset_index() 
df1 = df.groupby("d_from")['Hour'].max().reset_index() 
df1.index = df1['d_from']
dicex['max_active_hour'] = df1['Hour']

In [86]:
# add feature: Transaction level (total number orders for each customer count)
df = dicex_dataset.groupby('d_from')['trx_timestamp'].count().reset_index() 
df.index = df['d_from']
dicex['transaction_level'] = df['trx_timestamp']

In [57]:
# add feature: Active_hour_len
def continuous_hours(data):
    length = 1
    for i in range(len(data)-1):
        if data[i+1] == (data[i]+1):
            length += 1
        else:
            break
    return length

avg_hour_len = []
for x in np.unique(dicex_dataset.d_from):
    time_len = []
    df = dicex_dataset[dicex_dataset.d_from==x]
    for y in np.unique(df.Day):
        hour = np.unique(df[dicex_dataset.Day==y]['Hour'].reset_index(drop=True))
        time_len.append(continuous_hours(hour))
    avg_hour_len.append(np.mean(time_len))
dicex['Active_hour_len'] = avg_hour_len

C:\Users\sissy\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


In [59]:
# add feature: Max_Active_hour_len
max_hour_len = []
for x in np.unique(dicex_dataset.d_from):
    time_len = []
    df = dicex_dataset[dicex_dataset.d_from==x]
    for y in np.unique(df.Day):
        hour = np.unique(df[dicex_dataset.Day==y]['Hour'].reset_index(drop=True))
        time_len.append(continuous_hours(hour))
    max_hour_len.append(max(time_len))
dicex['Max_hour_len'] = max_hour_len

C:\Users\sissy\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [493]:
#add label
dicex['label'] = dicex['user'].apply(lambda x: 1 if x in bot_account_new else 0)

In [88]:
# save full data set with all features
dicex.to_csv('dicex_cleaned.csv')